# Replicating the section of choi et. al 2010 titled:
# *Implementation of FSEOF on lycopene-producing E. coli*
Choi et. al 2010: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2869140/

In [ ]:
import sys
sys.path.append("..")

Construction of the lycopene-producing *E. coli*

In [2]:
import examples
import fseof_additions
import fseof_media
import dh5_alpha

#load model
model = dh5_alpha.ecoli_dh5a()

#add necessary genes (lycopene pathways)
fseof_additions.add_crtE(model)
fseof_additions.add_crtB(model)
fseof_additions.add_crtI(model)
fseof_additions.add_lyco_dem(model)

#set LB media
fseof_media.simulate_LB_media(model)





In [3]:
#check objective to see if matches the one presented in paper

#max theoretical lycopene flux
model.objective = "LYCOdem"
solution = model.optimize()
solution.objective_value


0.9657492354740086

In [4]:
#run fseof to cross-check with choi et al. paper
from cobra_fseof.fseof import fseof
results = fseof(model, 10, "LYCOdem", "BIOMASS_Ec_iJO1366_core_53p95M", "max")
print(results.scan)

FSEOF; Running FVA: 100%|██████████| 10/10 [03:59<00:00, 23.97s/it]

                              target  vmin      vmax  flux0     flux1  \
23CUMPtex                      False   0.0  0.000000    0.0  0.000000   
23DAPPAt2pp                    False   0.0  0.000000    0.0  0.000000   
23DAPPAtex                     False   0.0  0.000000    0.0  0.000000   
BIOMASS_Ec_iJO1366_WT_53p95M   False   0.0  0.000000    0.0  0.000000   
23PDE2pp                       False   0.0  0.000000    0.0  0.000000   
...                              ...   ...       ...    ...       ...   
UAG4Ei                         False   0.0  0.000000    0.0  0.000000   
UGCIAMH                        False   0.0  0.000000    0.0  0.000000   
ZCRTE                           True   0.0  1.738349    0.0  0.193150   
ZCRTB                           True   0.0  0.869174    0.0  0.096575   
ZCRTI                           True   0.0  0.869174    0.0  0.096575   

                                flux2     flux3     flux4     flux5     flux6  \
23CUMPtex                     0.00000  0.0

In [5]:
#print list of reactions that our fseof identiified
targets = zip(results.scan.index, results.scan.target)
targets = dict(targets)
lst = []
for target in targets:
    if targets[target]:
        lst.append(target)

print(lst)





['AKGDH', 'CDPMEK', 'CYTK1', 'DMATT', 'DXPRIi', 'DXPS', 'FBA3', 'FE3Ri', 'FESD1s', 'FESR', 'GRTT', 'IPDDI', 'IPDPS', 'MECDPDH5', 'MECDPS', 'MEPCT', 'MOX', 'PIt2rpp', 'PItex', 'PFK_3', 'NDPK3', 'PGI', 'PGMT', 'POR5', 'PYK', 'RPE', 'TKT1', 'TKT2', 'TPI', 'MECDPDH2', 'ZCRTE', 'ZCRTB', 'ZCRTI']


In [6]:
# choi et al. 2010 Supplemental Table 4A (All reactions that were identified by their fseof)

#TCA cycle reactions
#'FUM_rxn' not in model, so replaced it with 'FUM'
tableFourTCA = ['ACONT', 'CS', 'FUM', 'ICDHyr', 'MDH', 'SUCDli', 'SUCOAS', 'AKGDH', 'SUCD4']


# 'MECDPDH5' from table 4A, is replaced with 'MECDPDH2' and 'MECDPDH5' here
tableFourLyc = ['CDPMEK', 'DMATT', 'DXPRIi', 'DXPS','GRTT', 'IPDDI', 'IPDPS', 'MECDPDH5', 'MECDPDH2', 'MECDPS', 'MEPCT', 'ZCRTE', 'ZCRTB', 'ZCRTI']


#TODO: PFK_3==PFK (but theres also PFK in model...)
# FBA is replaced with 'FBA3' http://bigg.ucsd.edu/models/iEC1368_DH5a/genes/locus_2187
tableFourGlyc = ['FBA3', 'PFK_3', 'PGI', 'TPI']


#disclaimer: 'CACTP' is replaced by 'CRNabcpp' (since model did not have CACTP rxn)
#CAITP is replaced by CRNt8pp (since model did not have CAITP rxn)
tableFourOther = ['ADK1', 'ADK4', 'CYTK1', 'CRNabcpp', 'CRNt8pp', 'PPA', 'CO2t', 'H2Ot', 'PIt2rpp']




In [9]:
#Of the genes in supp file 4, how many was our algorithm able to identify?

tableFourMegalst = tableFourTCA + tableFourLyc + tableFourGlyc + tableFourOther
print("Total number of reactions that the choi et. al 2010 paper identified: " + str(len(tableFourMegalst)))

count = 0
for item in tableFourMegalst:
    if item in lst:
        count += 1

print("How many out of the total number of reactions that the choi et. al paper identified, did our FSEOF identify: " + str(count) + "/" + str(len(tableFourMegalst)))
print("Above value as a percentage: " + str(count/len(tableFourMegalst)))


count1 = 0
for item in tableFourLyc:
    if item in lst:
        count1 += 1

print("What percentage of the Lycopene biosynthetic pathways that choi et. al identified, did our fseof identify: " + str(count1/len(tableFourLyc)))




Total number of reactions that the choi et. al 2010 paper identified: 36
How many out of the total number of reactions that the choi et. al paper identified, did our FSEOF identify: 21/36
Above value as a percentage: 0.5833333333333334
What percentage of the Lycopene biosynthetic pathways that choi et. al identified, did our fseof identify: 1.0
